<a href="https://colab.research.google.com/github/Jorgecardetegit/DiseaseClassifier/blob/main/Malaria_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam

In [4]:
dataset, dataset_info = tfds.load('malaria', with_info=True,
                                  as_supervised=True,
                                  shuffle_files = True,
                                  split=['train'])

# Data Split

In [5]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

In [6]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO )

# Data Preprocessing

## Data Augmentation

In [7]:
IM_SIZE = 224
def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/.255, label

In [8]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)
train_dataset

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [9]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size = 8, reshuffle_each_iteration = True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [10]:
val_dataset = val_dataset.shuffle(buffer_size = 8, reshuffle_each_iteration = True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [21]:
model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters = 6, kernel_size = 5, strides = 1, padding = "valid", activation = "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size = 2, strides = 2),

    Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "valid", activation = "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size = 2, strides = 2),

    Flatten(),

    Dense(1000, activation = "relu"),
    BatchNormalization(),
    Dense(100, activation = "relu"),
    BatchNormalization(),
    Dense(1, activation = "sigmoid"),
])



In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       456       
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 53, 53, 16)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 44944)             0         
                                                                 
 dense (Dense)               (None, 1000)              4

In [26]:
model.compile(optimizer = Adam(learning_rate = 0.1),
              loss = BinaryCrossentropy(),
              metrics = "accuracy")

In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)

Epoch 1/5
689/689 [==============================] - 1782s 3s/step - loss: 0.5285 - accuracy: 0.7514 - val_loss: 0.9764 - val_accuracy: 0.6338
Epoch 2/5
689/689 [==============================] - 1755s 3s/step - loss: 0.3343 - accuracy: 0.8734 - val_loss: 0.8212 - val_accuracy: 0.5525
Epoch 3/5
314/689 [============>.................] - ETA: 15:07 - loss: 0.2231 - accuracy: 0.9295

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train_loss", "val_loss"])
plt.show()


# Model Evaluation and Testing

In [ ]:
test_dataset = test_dataset.batch(1)

In [ ]:
lenet_model.evaluate(test_dataset)

In [ ]:
model.predict(test_dataset.take(1)[0][0])

In [ ]:
def parasite_or_not(x):
  if(x<0.5):
    return str("P")
  else:
    return str("U")

In [ ]:
paraiste_or_not(model.predict(test_dataset.take(1)[0][0]))

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):

  ax = plt.subplot(3,3, i+1)
  plt.imshow(image[0])
  plt.title(str(parasite_or_not(label.numpy()[0])) + ":" + str(parasite_or_not(lente_model.predict(image)[0][0])))
  plt.axis("off")

# Saving and Loading from Google Drive

In [ ]:
drive.mount("/content/drive/")

In [ ]:
!cp -r /content/lenet /content/drive/MyDrive/lenet_colab/


In [ ]:
!cp -r /content/drive/MyDrive/lenet_colab/ /content/lenet_colab/

# Creating model with other ways that the Sequential API

## Functional API

In [ ]:
funct_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input_image")

x = Conv2D(filters = 6, kernel_size = 5, strides = 1, padding = "valid", activation = "relu")(func_input)
x = BatchNormalization()(x)
x = MaxPool2D (pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "valid", activation = "relu")(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size = 2, strides = 2)(x)

x = Flatten()(x)

x = Dense(1000, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)

lenet_model_func = Model(func_input, func_output, name = "Lenet_Model")

In [ ]:
lenet_model_func.summary()

In [ ]:
lenet_model_func.compile(optimizer = Adam(learning_rate = 0.01),
                         loss = BinaryCrossentropy(),
                         metrics = "accuracy")

In [ ]:
history = lenet_model.fit(train_dataset, validation_data = val_dataset, epochs = 5, verbose = 1)

# Feature Extractor

In [ ]:
funct_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input_image")

x = Conv2D(filters = 6, kernel_size = 5, strides = 1, padding = "valid", activation = "relu")(func_input)
x = BatchNormalization()(x)
x = MaxPool2D (pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 16, kernel_size = 5, strides = 1, padding = "valid", activation = "relu")(x)
x = BatchNormalization()(x)
output = MaxPool2D(pool_size = 2, strides = 2)(x)

func_output = Dense(1, activation = "sigmoid")(x)

lenet_model_func = Model(func_input, output, name = "Feature_Extractor")

In [ ]:
feature_extractor_model.summary()

In [ ]:
funct_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input_image")

x = feature_extractor_model(func_input)
x = Flatten()(x)

x = Dense(1000, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)

lenet_model_func = Model(func_input, func_output, name = "Lenet_Model")

In [ ]:
lenet_model_func.summary()